In [ ]:
from math import gcd
from functools import reduce

# For computing LCM
def lcm(a, b):
    return abs(a * b) // gcd(a, b)

# LCM of multiple numbers
def lcm_multiple(numbers):
    return reduce(lcm, numbers)

# Task class to represent each task
class Task:
    def __init__(self, execution_time, period, m, k):
        self.e = execution_time  # execution time
        self.p = period  # period
        self.m = m  # weakly-hard constraint
        self.k = k  # weakly-hard constraint
        self.jobs = []  # Jobs will be generated later
        self.acceptance_pattern = []  # E-pattern

# Generate jobs based on the period and hyper-period
def generate_jobs(tasks):
    hyper_period = lcm_multiple([task.p for task in tasks])
    for task in tasks:
        num_jobs = hyper_period // task.p
        task.jobs = [(i * task.p, (i + 1) * task.p) for i in range(num_jobs)]
        generate_e_pattern(task)


#e-pattern
def generate_e_pattern(task):
    m, k = task.m, task.k
    pattern = []

    # Generate the initial E-pattern using the original formula logic
    for j in range(k):
        if j == (j * m) // k * (k // m):
            pattern.append(1)
        else:
            pattern.append(0)

    # Check if the generated pattern satisfies the (m, k) constraint
    if sum(pattern) < m:
        # If not satisfied, assign exactly `m` jobs as mandatory (1) and the rest as 0
        pattern = [1] * m + [0] * (k - m)

    # Extend the pattern to match the number of jobs in the hyper-period
    num_jobs = len(task.jobs)
    task.acceptance_pattern = pattern * (num_jobs // k) + pattern[:num_jobs % k]

# First-fit bin packing algorithm considering e/p instead of just execution time
def first_fit(tasks, processors, demand_function):
    # Sort tasks by increasing order of e/p ratio
    tasks = sorted(tasks, key=lambda task: task.e / task.p)

    for task in tasks:
        assigned = False
        for proc in processors:
            if demand_function(proc, task):
                proc.append(task)
                assigned = True
                break
        if not assigned:
            processors.append([task])

# Demand-based condition for EDF feasibility, considering e/p
def demand_function(processor, task):
    for t1, t2 in task.jobs:
        demand = sum(t.e / t.p for t in processor if t1 <= t.p and t.p <= t2)
        if demand + task.e / task.p > (t2 - t1) / task.p:
            return False
    return True

# E-pattern acceptance check
def check_e_pattern(task):
    # Check sliding windows of size k for E-pattern
    num_jobs = len(task.acceptance_pattern)
    for i in range(num_jobs - task.k + 1):
        window = task.acceptance_pattern[i:i + task.k]
        if sum(window) < task.m:
            return False  # The task doesn't satisfy the (m, k) constraint in this window
    return True

# Function to read tasks from a file
def read_tasks_from_file(filename):
    tasks = []
    with open(filename, 'r') as file:
        for line in file:
            if line.strip():  # Ignore empty lines
                e, p, m, k = map(int, line.split())
                tasks.append(Task(e, p, m, k))
    return tasks

# Main function to schedule tasks and find minimum processors
def schedule_tasks(tasks):
    processors = []  # List of processors
    generate_jobs(tasks)  # Generate jobs for the tasks
    for task in tasks:
        if not check_e_pattern(task):
            print(f"Task with period {task.p} does not satisfy the (m, k) acceptance pattern.")
            return None  # Task doesn't meet (m, k) constraint
    first_fit(tasks, processors, demand_function)  # Use first-fit bin packing (with sorting by e/p)
    return len(processors)  # Return number of processors used

# Read tasks from the input file
tasks = read_tasks_from_file('input.txt')

# Schedule the tasks
min_processors = schedule_tasks(tasks)
if min_processors is not None:
    print("Minimum number of processors required:", min_processors)


Minimum number of processors required: 4


Function LCM(a, b):
    Return abs(a * b) // GCD(a, b)

Function LCM_multiple(numbers):
    Return Reduce(LCM, numbers)

Class Task:
    Initialize with execution_time (e), period (p), m, k
    Create empty list for jobs
    Create empty list for acceptance_pattern

Function generate_jobs(tasks):
    hyper_period = LCM_multiple([task.p for task in tasks])
    For each task in tasks:
        num_jobs = hyper_period // task.p
        Generate jobs as [(i * task.p, (i + 1) * task.p) for i in range(num_jobs)]
        Call generate_e_pattern(task)

Function generate_e_pattern(task):
    m, k = task.m, task.k
    Initialize pattern as empty list
    For each job j in range(k):
        If j == (j * m) // k * (k // m):
            Append 1 to pattern (mandatory job)
        Else:
            Append 0 to pattern (non-mandatory job)
    Extend pattern to match the number of jobs

Function first_fit(tasks, processors, demand_function):
    Sort tasks by increasing e/p ratio
    For each task in tasks:
        assigned = False
        For each processor in processors:
            If demand_function(processor, task) is True:
                Assign task to processor
                Set assigned to True
                Break
        If not assigned:
            Add new processor with task

Function demand_function(processor, task):
    For each job interval (t1, t2) in task.jobs:
        Calculate total demand as sum of e/p for tasks in processor within interval
        If demand + e/p of task exceeds capacity of processor:
            Return False
    Return True

Function check_e_pattern(task):
    For each window of size k in task.acceptance_pattern:
        If sum(window) < task.m:
            Return False (doesn't meet (m, k) constraint)
    Return True

Function read_tasks_from_file(filename):
    Initialize empty list for tasks
    Open file and read each line:
        Parse e, p, m, k from the line
        Create new Task with e, p, m, k and add to tasks list
    Return tasks list

Function schedule_tasks(tasks):
    Initialize empty list for processors
    Call generate_jobs(tasks)
    For each task in tasks:
        If check_e_pattern(task) is False:
            Print task doesn't satisfy (m, k) constraint
            Return None
    Call first_fit(tasks, processors, demand_function)
    Return length of processors

# Main program
tasks = read_tasks_from_file('input.txt')
min_processors = schedule_tasks(tasks)
If min_processors is not None:
    Print "Minimum number of processors required: " + min_processors
